In [61]:
from ImageProcessor import Frame
from deuces import Card
from deuces import Evaluator
import time
import os
from collections import Counter
from IPython.display import clear_output
import numpy as np

In [100]:
frame = Frame(blinds = .05)

In [113]:
playOn = True
ai = AI()

while playOn: 
    frame.update()
    
    while frame.identify_card('hole1') is 'None' or frame.identify_card('hole2') is 'None':
        print 'no cards found'
        time.sleep(3)
        frame.update()
        
    c1 = frame.identify_card('hole1')
    c2 = frame.identify_card('hole2') 
    
    Card.print_pretty_cards([Card.new(c1), Card.new(c2)])
    #print c1, c2
    
    ## check to see if we should fold the cards immediately
    if frame.user_position() != 'BB' and ai.initial_screening(c1, c2) is False:          
        print 'FOLDING'
        frame.fold()
        time.sleep(.5)
        frame.fold()
        #clear_output()
        time.sleep(8)
        
        
    else:    
        ##wait until user turn
        if frame.user_position() is 'BB':
            time.sleep(3)
            frame.update()
        
        #print 'postion' , frame.user_position()
        print 'is turn? : ' , frame.is_user_turn()


        while(not frame.is_user_turn()):
            print frame.is_user_turn()
            time.sleep(3)
            frame.update()
            #print frame.user_position()
    
        
        ## BOT MAKES MOVE
        bet = ai.calculate_bet(frame, c1, c2)
        
        print bet
        
        if bet == 'fold':
            frame.fold()
            time.sleep(4)
            continue
        elif bet == 'call':
            frame.call()
            
                 
        else:
            frame.bet(bet)
         
        #print 'BOT MAKES DECISION HERE'
        os.system("say 'Your turn'")
        
        
        break
    
    
    

  [ K ❤ ] , [ A ♦ ]  
is turn? :  True
position: SB
pot 0.22
0  limpers
1 raisers
0 callers
CO  raiser position
call


In [114]:
class AI():
    
    
    def __init__(self):
        '''If our hand is not in this list, fold IMMEDIATELY (unless big blind)'''
        self.pairs_1 = ['AA', 'KK', 'QQ']
        self.pairs_2 = ['JJ', 'TT', '99'] 
        self.pairs_3 = ['88', '77']
        self.pairs_4 = ['66', '55', '44', '33', '22']
        self.pairs = self.pairs_1 + self.pairs_2 + self.pairs_3 + self.pairs_4

        self.suited_connectors_1 = ['KQs', 'JQs', 'JTs']
        self.suited_connectors_2 = ['9Ts', '89s', '78s', '67s']
        self.suited_connectors_3 = ['56s', '45s', '34s', '23s']
        self.suited_connectors = self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_3

        self.suited_connectors_one_gap_1 = ['JKs', 'QTs', '9Js']
        self.suited_connectors_one_gap_2 = ['8Ts', '79s', '68s']
        self.suited_connectors_one_gap_3 = ['57s', '46s', '35s', '24s']
        self.suited_connectors_one_gap = self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2 + self.suited_connectors_one_gap_3

        self.suited_connectors_two_gap_1 = ['KTs', 'Q9s', 'J8s']
        self.suited_connectors_two_gap_2 = ['7Ts', '69s', '58s']
        self.suited_connectors_two_gap_3 = ['47s', '36s', '25s']
        self.suited_connectors_two_gap = self.suited_connectors_two_gap_1 + self.suited_connectors_two_gap_2 + self.suited_connectors_two_gap_3

        self.suited_aces_1 = ['AJs', 'ATs']
        self.suited_aces_2 = ['6As', '7As', '8As', '9As']
        self.suited_aces_3 = ['2As', '3As', '4As', '5As']
        self.suited_aces = self.suited_aces_1 + self.suited_aces_2 + self.suited_aces_3 

        self.big_aces = ['AKs', 'AK', 'AQ', 'AQs']
        
        self.play_list = self.pairs + self.suited_connectors + self.suited_connectors_one_gap + \
                         self.suited_connectors_two_gap + self.suited_aces  + self.big_aces
                   
        
    
    
    def encode_hand(self, c1, c2):
        if c1[1] == c2[1]:
            encoded_hand = c1[0]+c2[0]+'s'
        else:
            encoded_hand = c1[0]+c2[0] 
        return ''.join(sorted(encoded_hand))
    
    def initial_screening(self, c1, c2):
        '''Returns false if we should fold hand immediately'''
        hand = self.encode_hand(c1, c2)
        return hand in self.play_list
    
    def num_raisers(self, bets, max_bet = 0):
        raisers = 0
        for bet in bets:
            if bet > max_bet:
                raisers+= 1
                max_bet = bet
        return raisers
    def num_limpers(self, bets, position, blinds):
        num_limpers = len(bets[bets == blinds]) - 1  
        if position is 'BB':
            num_limpers += 1
        return num_limpers
    
    def is_unopened(self, position, bets, blinds):
        return position != 'BB' and len(bets[bets >= blinds]) == 1
    def is_limpers(self, bets, blinds):
        return bets.max() <= blinds
    def raiser_position(self, bets,position):   
        positions = ['BTN', 'SB', 'BB','UTG', 'MP', 'CO']
        user_index = positions.index(position)
        return positions[(user_index + np.argmax(bets) + 1)%6]
    def bet_by_position(self, bets, our_position, opp_position):
        positions = ['BTN', 'SB', 'BB','UTG', 'MP', 'CO']
        user_index = positions.index(our_position)
        bet_index = positions.index(opp_position)
        if user_index <= bet_index:
            loc = bet_index-user_index
        else:
            loc = ((6 - user_index) + bet_index)%6
        return bets[loc - 1]
           
    
    
    def calculate_bet(self, frame, c1, c2):
        hand = self.encode_hand(c1, c2)
        bets = frame.bets()
        pot = float(frame.read_value('pot'))
        blinds = frame.blinds
        position = frame.user_position()
        
        print 'position:' , position
        print 'pot', pot
        print self.num_limpers(bets, position, blinds) , ' limpers'
        print self.num_raisers(bets, max_bet = blinds) , 'raisers'
        print max(Counter(bets).most_common())[1] - 1, 'callers'
        print self.raiser_position(bets, position) , ' raiser position'
        
        
        if position is 'UTG':
            if hand in self.pairs_1 + self.pairs_2 + self.pairs_3 + self.big_aces:
                    return 3*blinds
            elif hand in  self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2 \
                        + self.suited_connectors_one_gap_2 + self.suited_aces_3 + self.suited_aces_1:
                    return 2*blinds
        if position is 'MP':
            if self.is_unopened(position, bets, blinds):
                if hand in self.pairs_1 + self.pairs_2 + self.pairs_3 + self.suited_connectors_1 \
                   + self.suited_connectors_2 + self.suited_aces_1 + self.big_aces:
                    return 3*blinds
                
                elif hand in self.pairs_4 + self.suited_connectors_2 + self.suited_connectors_one_gap + self.suited_aces_3:
                    return 2*blinds
            elif self.is_limpers(bets, blinds):
                if hand in self.pairs_1:
                    return 6*blinds
                elif hand in ['AKs', 'JJ']:
                    return 5*blinds
                elif hand in ['TT', '99', 'AK', 'AQs']:
                    return 4*blinds
                elif hand in self.pairs_3 + self.suited_aces_1:
                    return 3*blinds
                elif self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_one_gap_1\
                   + self.suited_connectors_one_gap_2 + self.suited_aces_3:
                    return 'call'
            else: #UTG must have raised
                villan_bet = bets.max()
                if villan_bet >= 5*blinds:
                    if hand in self.pairs_1:
                        return 2*pot
                elif villan_bet >= 4*blinds:
                    if hand in self.pairs_1:
                        return 2*pot
                    if hand in self.pairs_2 + self.big_aces:
                        return 'call'
                elif villan_bet >= 3*blinds:
                    if hand in self.pairs_1 + ['AKs', 'AK']:
                        return 2*pot
                    if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_1\
                               + self.suited_connectors_2  + self.suited_connectors_one_gap_1 +\
                               + self.suited_connectors_one_gap_2 + self.big_aces:
                        return 'call'
                else:
                    if hand in self.pairs_1 + self.pairs_2 + self.big_aces:
                        return 2*pot
                    if hand in self.pairs_3 + self.pairs_4 + self.suited_connectors_1  + self.suited_connectors_2\
                               + self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2\
                               + self.suited_aces_1 + self.suited_aces_3:
                        return 'call'
        if position is 'CO':
            if self.is_unopened(position, bets, blinds):
                if self.pairs + self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_one_gap_1\
                    + self.suited_connectors_one_gap_2 + self.suited_aces_1 + self.suited_connectors_3 + self.big_aces:
                    return 3*blinds
                
            elif self.is_limpers(bets, blinds):
                num_limpers = self.num_limpers(bets, position, blinds)
                if num_limpers is 1:         
                    if hand in self.pairs_1:
                        return 6*blinds
                    if hand in ['AKs', 'JJ']:
                        return 5*blinds
                    if hand in ['TT', '99', 'AK', 'AQs']:
                        return 4*blinds
                    if hand in self.pairs_3 + self.suited_aces_1:
                        return 3*blinds
                    if self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_one_gap_1\
                       + self.suited_connectors_one_gap_2 + self.suited_aces_3:
                        return 'call'
                if num_limpers is 2:
                    if hand in self.pairs_1:
                        return 8*blinds
                    if hand in self.pairs_2 + self.big_aces:
                        return 6*blinds
                    if hand in  self.pairs_3 + self.pairs_4 + self.suited_connectors + self.suited_connectors_one_gap\
                                + self.suited_connectors_two_gap + self.suited_aces:
                        return 'call'
            else: #Must be raiser
                num_raisers = self.num_raisers(bets, max_bet = blinds)
                villan_bet = bets.max()
                if num_raisers is 1:
                    num_callers = max(Counter(bets).most_common())[1] - 1       
                    if num_callers is 0:
                        if villan_bet >= 5*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                        elif villan_bet >= 4*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                            if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                        + self.suited_connectors_one_gap_2 + ['AKs', 'AK']:
                                return 'call'
                        elif villan_bet >= 3*blinds:
                            if hand in self.pairs_1 + ['AKs', 'AK']:
                                return 2*pot
                            if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_1\
                                       + self.suited_connectors_2  + self.suited_connectors_one_gap_1 +\
                                       + self.suited_connectors_one_gap_2 + self.big_aces:
                                return 'call'
                        else:
                            if hand in self.pairs_1 + self.pairs_2 + self.big_aces:
                                return 2*pot
                            if hand in self.pairs_3 + self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2\
                                       + self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2\
                                       + self.suited_aces_1 + self.suited_aces_3:
                                return 'call'                  
                    elif num_callers is 1:
                        if villan_bet >= 7*blinds:
                            if hand in self.pairs_1:
                                return 3*pot
                        elif villan_bet >= 5*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                            if hand is 'AKs':
                                return 'call'         
                        elif villan_bet >= 4*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                            if hand in self.pairs_2 + self.big_aces:
                                return 'call'
                        elif villan_bet >= 3*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                            if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                       + self.suited_connectors_one_gap_2 + self.big_aces:
                                return 'call'
                        else:
                            if hand in self.pairs_1 + ['AKs', 'AK']:
                                return 2*pot
                            if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors\
                                       + self.suited_connectors_one_gap + self.suited_connectors_two_gap_2\
                                       + self.suited_aces + ['AQs', 'AQ']:
                                return 'call'
                if num_raisers >= 2:
                    if villan_bet > 12*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                    elif villan_bet > 9*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                        if hand is 'AKs':
                            return 'call'
                        
                    elif villan_bet >= 6*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                        if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                    + ['AKs', 'AK']:
                            return 'call'
                    elif villan_bet >= 3*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                        if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                    + self.suited_connectors_one_gap_2 + ['AKs', 'AK']:
                            return 'call'
                        
                    
                    
                    
                       
        if position is 'BTN':
            if self.is_unopened(position, bets, blinds):
                if self.pairs + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                    + self.suited_aces + self.big_aces:
                    return 3*blinds
                
            elif self.is_limpers(bets, blinds):
                num_limpers = self.num_limpers(bets, position, blinds)
                if num_limpers is 1:         
                    if hand in self.pairs_1:
                        return 5*blinds
                    if hand in self.pairs_2 + self.big_aces:
                        return 4*blinds
                    if hand in self.pairs_3 + self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2\
                                + self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2 + self.suited_aces_1:
                        return 3*blinds
                    if hand in self.pairs_3 + self.suited_aces_1:
                        return 3*blinds
                    if hand in self.suited_connectors_3 + self.suited_connectors_one_gap_3 + self.suited_connectors_two_gap\
                               + self.suited_aces_2 + self.suited_aces_3:
                        return 'call'
                if num_limpers is 2:
                    if hand in self.pairs_1:
                        return 7*blinds
                    if hand in self.pairs_2 + self.big_aces:
                        return 5*blinds
                    if hand in  self.pairs_3 + self.pairs_4 + self.suited_connectors + self.suited_connectors_one_gap\
                                + self.suited_connectors_two_gap + self.suited_aces:
                        return 'call'
                if num_limpers is 3:
                    if hand in self.pairs_1:
                        return 10*blinds
                    if hand in self.pairs_2 + self.big_aces:
                        return 6*blinds
                    if hand in  self.pairs_3 + self.pairs_4 + self.suited_connectors + self.suited_connectors_one_gap\
                                + self.suited_connectors_two_gap + self.suited_aces:
                        return 'call'
            else: #Must be raiser
                num_raisers = self.num_raisers(bets, max_bet = blinds)
                villan_bet = bets.max()
                
                if num_raisers is 1:
                    num_callers = max(Counter(bets).most_common())[1] - 1
                    
                    if self.bet_by_position(bets, position, 'UTG') > blinds or self.bet_by_position(bets, position, 'MP') > blinds:    
                        if num_callers is 0:
                            if villan_bet > 5*blinds:
                                if hand in self.pairs_1:
                                    return 2*pot
                            elif villan_bet >= 4*blinds:
                                if hand in self.pairs_1:
                                    return 2*pot
                                if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                            +['AK', 'AKs']:
                                    return 'call'

                            elif villan_bet >= 3*blinds:
                                if hand in self.pairs_1 + ['AKs']:
                                    return 2*pot
                                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors\
                                            + self.suited_connectors_one_gap + self.suited_connectors_two_gap_1\
                                            +  self.suited_connectors_two_gap_2 + self.big_aces:
                                    return 'call'
                            else:
                                if hand in self.pairs_1 + self.pairs_2 + self.big_aces:
                                    return 2*pot
                                else:
                                    return 'call'
                        if num_callers > 0:
                            if villan_bet > 5*blinds:
                                if hand in self.pairs_1:
                                    return 2*pot
                            elif villan_bet >= 4*blinds:
                                if hand in self.pairs_1:
                                    return 2.5*pot
                                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_1\
                                            + self.suited_connectors_2 + self.suited_connectors_3\
                                            + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                                            + self.suited_aces_3 + ['AK', 'AKs', 'AQs']:
                                    return 'call'

                            elif villan_bet >= 3*blinds:
                                if hand in self.pairs_1 + ['AKs', 'JJ']:
                                    return 2.5*pot
                                else:
                                    return 'call'
                            else:
                                if hand in self.pairs_1 + ['AKs', 'JJ']:
                                    return 2.5*pot
                                if hand in ['AK', 'AQs']:
                                    return 1.75*pot
                                else:
                                    return 'call'
                                        
                    else: #cutoff raiser
                        if villan_bet >= 5*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                        elif villan_bet >= 4*blinds:
                            if hand in self.pairs_1:
                                return 2*pot
                            if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                        + ['AK', 'AKs']:
                                return 'call'
                        elif villan_bet >= 3*blinds:
                            if hand in self.pairs_1 + self.pairs_2 + ['AKs', 'AK']:
                                return 2*pot
                            if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors\
                                        + self.suited_connectors_one_gap + self.suited_connectors_two_gap_1\
                                        + self.suited_connectors_two_gap_2 + self.big_aces:
                                return 'call'
                        else:
                            if hand in self.pairs_1 + self.pairs_2 + self.pairs_3 + self.big_aces\
                                        + self.suited_connectors_1 +  self.suited_connectors_2 + self.suited_connectors_one_gap_2:
                                return 2*pot
                            else:
                                return 'call'
                
                if num_raisers >= 2:
                    if villan_bet > 12*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                    elif villan_bet > 9*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                        if hand is 'AKs':
                            return 'call'
                        
                    elif villan_bet >= 6*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                        if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                    + ['AKs', 'AK']:
                            return 'call'
                    elif villan_bet >= 3*blinds:
                        if hand in self.pairs_1:
                            return 2.5*pot
                        if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                    + self.suited_connectors_one_gap_2 + ['AKs', 'AK']:
                            return 'call'
                        
                
                    
                    
            
        if position is 'SB':
            if self.is_unopened(position, bets, blinds):
                if hand in self.pairs + self.suited_connectors_1 + self.suited_connectors_2 \
                            + self.suited_connectors_one_gap_1 +  self.suited_connectors_one_gap_2\
                            + self.suited_connectors_two_gap_1  + self.suited_aces + self.big_aces:
                    return 3*blinds
                
            elif self.is_limpers(bets, blinds):
                num_limpers = self.num_limpers(bets, position, blinds)
                raise_amount = [0, 7, 10, 15, 20]
                if hand in self.pairs_1:
                    return raise_amount[num_limpers]*blinds
                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces:
                    return 'call'
            else: #Must be raiser
                num_raisers = self.num_raisers(bets, max_bet = blinds)
                villan_bet = bets.max()
                
                if num_raisers is 1:
                    num_callers = max(Counter(bets).most_common())[1] - 1
                    if self.bet_by_position(bets, position, 'UTG') > blinds or self.bet_by_position(bets, position, 'MP') > blinds or self.bet_by_position(bets, position, 'CO'):
                        if num_callers is 0:
                            if villan_bet >= 5*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                            elif villan_bet >= 4*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                                if hand in self.pairs_2 + ['AKs', 'AK']:
                                    return 'call'
                            else:
                                if hand in self.pairs_1 + ['AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                            + self.big_aces:
                                    return 'call'
                        if num_callers >= 1:
                            if villan_bet >= 7*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                            elif villan_bet >= 5*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                                if hand in self.pairs_2 + ['AKs']:
                                    return 'call'
                            elif villan_bet >= 4*blinds:
                                if hand in self.pairs_1 + ['JJ', 'AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2 +['AKs', 'AQs']:
                                    return 'call'
                            elif villan_bet >= 3*blinds:
                                if hand in self.pairs_1 + ['JJ','AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                            + self.suited_connectors_one_gap_2 + self.suited_aces:
                                    return 'call'
                            else:
                                if hand in self.pairs_1 + ['JJ', 'AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                            + self.suited_connectors_one_gap_2 + self.suited_aces:
                                    return 'call'
                            
                                
                    else: #button raised
                        if villan_bet >= 7*blinds:
                            if hand in self.pairs_1:
                                return 3.5*pot
                        elif villan_bet >= 5*blinds:
                            if hand in self.pairs_1:
                                return 3.5*pot
                            if hand is 'AKs':
                                return 'call'
                        elif villan_bet >= 4*blinds:
                            if hand in self.pairs_1 + ['JJ', 'AKs']:
                                return 3.5*pot
                            if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2 +['AKs', 'AQs']:
                                return 'call'
                        elif villan_bet >= 3*blinds:
                            if hand in self.pairs_1 + self.pairs_2 + ['AKs', 'AK']:
                                return 3.5*pot
                            if hand in self.suited_connectors_2  + self.suited_connectors_one_gap_2 + ['AQs','AQ']:
                                return 'call'
                        else:
                            if hand in self.pairs_1 + self.big_aces + self.pairs_2:
                                return 3.5*pot
                            if hand in self.pairs_3 + self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                            + self.suited_connectors_one_gap_1 + self.suited_aces_1:
                                return 'call'
                            
                if num_raisers >= 2: 
                    if villan_bet >= 12*blinds:
                        if hand in self.pairs_1:
                            return 3.5*pot
                    elif villan_bet > 9*blinds:
                        if hand in self.pairs_1:
                            return 3.5*pot
                        if hand in ['AKs','JJ']:
                            return 'call'
                        
                    elif villan_bet >= 6*blinds:
                        if hand in self.pairs_1:
                            return 3.5*pot
                        if hand in self.pairs_2 + ['AKs']:
                            return 'call'
                    elif villan_bet >= 3*blinds:
                        if hand in self.pairs_1 + ['JJ', 'AKs', 'AK']:
                            return 3.5*pot
                        if hand in self.pairs_2 +self.pairs_3 + self.suited_connectors_2\
                                    + self.suited_connectors_one_gap_2 + ['AQs']:
                            return 'call'
        
        if position is 'BB':
            if self.is_limpers(bets, blinds):
                num_limpers = self.num_limpers(bets, position, blinds)
                raise_amount = [0, 7, 10, 15, 20, 25]
                if hand in self.pairs_1:
                    return raise_amount[num_limpers]*blinds
                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces:
                    return 'call'
                
            else: #Must be raiser
                num_raisers = self.num_raisers(bets, max_bet = blinds)
                villan_bet = bets.max()
                
                if num_raisers is 1:
                    num_callers = max(Counter(bets).most_common())[1] - 1
                    if self.bet_by_position(bets, position, 'UTG') > blinds or self.bet_by_position(bets, position, 'MP') > blinds or self.bet_by_position(bets, position, 'CO'):
                        if num_callers is 0:
                            if villan_bet >= 5*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                            elif villan_bet >= 4*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                                if hand in self.pairs_2 + ['AKs', 'AK']:
                                    return 'call'
                            else:
                                if hand in self.pairs_1 + ['AKs']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                            + self.big_aces:
                                    return 'call'
                        if num_callers >= 1:
                            if villan_bet >= 7*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                            elif villan_bet >= 5*blinds:
                                if hand in self.pairs_1:
                                    return 3.5*pot
                                if hand in self.pairs_2 + ['AKs']:
                                    return 'call'
                            elif villan_bet >= 4*blinds:
                                if hand in self.pairs_1 + ['JJ', 'AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2 +['AKs', 'AQs']:
                                    return 'call'
                            elif villan_bet >= 3*blinds:
                                if hand in self.pairs_1 + ['JJ','AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 +  self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                            + self.suited_connectors_one_gap_2 + self.suited_aces:
                                    return 'call'
                            else:
                                if hand in self.pairs_1 + ['JJ', 'AKs', 'AK']:
                                    return 3.5*pot
                                if hand in self.pairs_2 + self.pairs_3 + self.pairs_4 + self.suited_connectors_2\
                                            + self.suited_connectors_one_gap_2 + self.suited_aces:
                                    return 'call'
                            
                                
                    elif self.bet_by_position(bets, position, 'BTN') > blinds: #button raised
                        if villan_bet >= 7*blinds:
                            if hand in self.pairs_1:
                                return 3.5*pot
                        elif villan_bet >= 5*blinds:
                            if hand in self.pairs_1:
                                return 3.5*pot
                            if hand is 'AKs':
                                return 'call'
                        elif villan_bet >= 4*blinds:
                            if hand in self.pairs_1 + ['JJ', 'AKs']:
                                return 3.5*pot
                            if hand in self.pairs_2 + self.suited_connectors_2 + self.suited_connectors_one_gap_2 +['AKs', 'AQs']:
                                return 'call'
                        elif villan_bet >= 3*blinds:
                            if hand in self.pairs_1 + self.pairs_2 + ['AKs', 'AK']:
                                return 3.5*pot
                            if hand in self.suited_connectors_2  + self.suited_connectors_one_gap_2 + ['AQs','AQ']:
                                return 'call'
                        else:
                            if hand in self.pairs_1 + self.big_aces + self.pairs_2:
                                return 3.5*pot
                            if hand in self.pairs_3 + self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                            + self.suited_connectors_one_gap_1 + self.suited_aces_1:
                                return 'call'
                    else: #SB raised
                        if villan_bet >= 10*blinds:
                            if hand in self.pairs_1:
                                return 2.5*pot
                        elif villan_bet >= 5*blinds:
                            if hand in self.pairs_1:
                                return 2.5*pot
                            if hand in ['AKs', 'AK', 'JJ', 'TT']:
                                return 'call'
                        elif villan_bet >= 4*blinds:
                            if hand in self.pairs_1:
                                return 2.5*pot
                            if hand in self.pairs_2 + self.pairs_3 + self.suited_connectors_1\
                                        + self.suited_connectors_2 + self.suited_connectors_one_gap_2\
                                        + self.suited_connectors_one_gap_1 + self.suited_aces_1 + self.big_aces:
                                return 'call'
                        else:
                            if hand in self.pairs_1 + self.big_aces + self.pairs_2:
                                return 2.5*pot
                            if hand in self.play_list:
                                return 'call'
                        
                            
                if num_raisers >= 2: 
                    if villan_bet >= 12*blinds:
                        if hand in self.pairs_1:
                            return 3.5*pot
                    elif villan_bet > 9*blinds:
                        if hand in self.pairs_1:
                            return 3.5*pot
                        if hand in ['AKs','JJ']:
                            return 'call'
                        
                    elif villan_bet >= 6*blinds:
                        if hand in self.pairs_1:
                            return 3.5*pot
                        if hand in self.pairs_2 + ['AKs']:
                            return 'call'
                    elif villan_bet >= 3*blinds:
                        if hand in self.pairs_1 + ['JJ', 'AKs', 'AK']:
                            return 3.5*pot
                        if hand in self.pairs_2 + self.pairs_3 + self.suited_connectors_2\
                                    + self.suited_connectors_one_gap_2 + ['AQs']:
                            return 'call'
        
            
                  
        return 'fold'
                
        
        
    
    
    
    
    
    
    